In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date

# Initialize Spark session
spark = SparkSession.builder.appName("ETL_FactTable").getOrCreate()

# Load SalesOrderHeader and SalesOrderDetail from local paths in DBFS
sales_order_header_df = spark.read.csv("dbfs:/FileStore/tables/production/salesorderheader.csv", header=True, inferSchema=True) \
    .filter(col("OnlineOrderFlag") == 1) \
    .select(
        col("SalesOrderID"),
        col("SalesOrderNumber"),
        to_date(col("OrderDate")).alias("OrderDate"),
        col("CustomerID"),
        col("TerritoryID"),
        col("TaxAmt"),
        col("Freight")
    )

sales_order_detail_df = spark.read.csv("dbfs:/FileStore/tables/production/salesorderdetails.csv", header=True, inferSchema=True)
product_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/tables/production/Production_product.csv")


In [0]:
# Sort data by SalesOrderID
sales_order_header_sorted = sales_order_header.orderBy("SalesOrderID")
sales_order_detail_sorted = sales_order_detail.orderBy("SalesOrderID")

# Merge Join based on SalesOrderID
merged_data = sales_order_header_sorted.join(
    sales_order_detail_sorted,
    "SalesOrderID",
    "inner"
)


In [0]:
# Load product dimension data
product_dim = spark.read.csv("dbfs:/FileStore/tables/dimension/dim_product_sale", header=True, inferSchema=True)

# 9.3 Lookup Transformation - Add product_sk from product_df based on ProductID
merged_data_with_product_sk = merged_data.join(
    product_df.select("productID", "product_sk"),
    merged_data.ProductID == product_df.product_id,
    "left"
).drop(product_df.product_id)

# 9.4 DRC - Replace nulls
merged_data_validated = merged_data_with_product_sk \
    .withColumn("TaxAmt", coalesce(col("TaxAmt"), lit(0))) \
    .withColumn("Freight", coalesce(col("Freight"), lit(0))) \
    .withColumn("product_sk", coalesce(col("product_sk"), lit(-1)))  # Use -1 for missing product_sk

# 9.5 DRC - Add additional fields like extended_sales and extended_cost
merged_data_final = merged_data_validated \
    .withColumn("extended_sales", col("OrderQty") * col("UnitPrice")) \
    .withColumn("extended_cost", col("OrderQty") * col("standard_cost"))


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2715838316518987>:6
      2 product_dim = spark.read.csv("dbfs:/FileStore/tables/dimension/dim_product_sale", header=True, inferSchema=True)
      4 # 9.3 Lookup Transformation - Add product_sk from product_df based on ProductID
      5 merged_data_with_product_sk = merged_data.join(
----> 6     product_df.select("productID", "product_sk"),
      7     merged_data.ProductID == product_df.product_id,
      8     "left"
      9 ).drop(product_df.product_id)
     11 # 9.4 DRC - Replace nulls
     12 merged_data_validated = merged_data_with_product_sk \
     13     .withColumn("TaxAmt", coalesce(col("TaxAmt"), lit(0))) \
     14     .withColumn("Freight", coalesce(col("Freight"), lit(0))) \
     15     .withColumn("product_sk", coalesce(col("product_sk"), lit(-1)))  # Use -1 for missing product_sk

File /databricks/spark/pytho

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, coalesce, lit, to_date, expr

# Initialize Spark session
spark = SparkSession.builder.appName("FactTableETL").getOrCreate()

# Load the data into DataFrames
#product_df = spark.read.option("header", "true").csv("/dbfs/FileStore/tables/dim_product_table.csv", inferSchema=True)
#sales_order_header_df = spark.read.option("header", "true").csv("/dbfs/FileStore/tables/SalesOrderHeader.csv", inferSchema=True)
#sales_order_detail_df = spark.read.option("header", "true").csv("/dbfs/FileStore/tables/SalesOrderDetail.csv", inferSchema=True)

# 9.1 Extract Phase - Filter online sales from SalesOrderHeader
sales_order_header_filtered = sales_order_header_df.filter(col("OnlineOrderFlag") == 1) \
    .select(
        col("SalesOrderID"),
        col("SalesOrderNumber"),
        to_date(col("OrderDate")).alias("OrderDate"),
        col("CustomerID"),
        col("TerritoryID"),
        col("TaxAmt"),
        col("Freight")
    )

# 9.2 Transform Phase - Sort and Merge Join on SalesOrderID
# Sort both dataframes by SalesOrderID
sales_order_header_sorted = sales_order_header_filtered.orderBy("SalesOrderID")
sales_order_detail_sorted = sales_order_detail_df.orderBy("SalesOrderID")

# Merge join on SalesOrderID
merged_data = sales_order_header_sorted.join(
    sales_order_detail_sorted,
    "SalesOrderID",
    "inner"
)

# 9.3 Lookup Transformation - Add product_sk from product_df based on ProductID
merged_data_with_product_sk = merged_data.join(
    product_df.select("product_id", "product_sk"),
    merged_data.ProductID == product_df.product_id,
    "left"
).drop(product_df.product_id)

# 9.4 DRC - Replace nulls
merged_data_validated = merged_data_with_product_sk \
    .withColumn("TaxAmt", coalesce(col("TaxAmt"), lit(0))) \
    .withColumn("Freight", coalesce(col("Freight"), lit(0))) \
    .withColumn("product_sk", coalesce(col("product_sk"), lit(-1)))  # Use -1 for missing product_sk

# 9.5 DRC - Add additional fields like extended_sales and extended_cost
merged_data_final = merged_data_validated \
    .withColumn("extended_sales", col("OrderQty") * col("UnitPrice")) \
    .withColumn("extended_cost", col("OrderQty") * col("standard_cost"))

# 9.6 Load Phase - Select columns for the fact table
fact_table = merged_data_final.select(
    col("product_sk"),
    col("SalesOrderID").alias("sales_order_id"),
    col("SalesOrderDetailID").alias("line_number"),
    col("OrderQty").alias("quantity"),
    col("UnitPrice").alias("unit_price"),
    col("standard_cost").alias("unit_cost"),
    col("TaxAmt").alias("tax_amount"),
    col("Freight").alias("freight"),
    col("extended_sales"),
    col("OrderDate").alias("created_at"),
    col("extended_cost")
)

# Writing to a Fact Table - Replace placeholders with appropriate details
fact_table.write.format("jdbc").options(
    url="jdbc:sqlserver://<server>;database=<database>",
    dbtable="FactSalesTable",
    user="<user>",
    password="<password>"
).mode("append").save()

# Check the result
fact_table.show(5)


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2715838316518990>:38
     30 merged_data = sales_order_header_sorted.join(
     31     sales_order_detail_sorted,
     32     "SalesOrderID",
     33     "inner"
     34 )
     36 # 9.3 Lookup Transformation - Add product_sk from product_df based on ProductID
     37 merged_data_with_product_sk = merged_data.join(
---> 38     product_df.select("product_id", "product_sk"),
     39     merged_data.ProductID == product_df.product_id,
     40     "left"
     41 ).drop(product_df.product_id)
     43 # 9.4 DRC - Replace nulls
     44 merged_data_validated = merged_data_with_product_sk \
     45     .withColumn("TaxAmt", coalesce(col("TaxAmt"), lit(0))) \
     46     .withColumn("Freight", coalesce(col("Freight"), lit(0))) \
     47     .withColumn("product_sk", coalesce(col("product_sk"), lit(-1)))  # Use -1 for missing product_

In [0]:
# Merge join on SalesOrderID
merged_data = sales_order_header_sorted.join(
    sales_order_detail_sorted,
    "SalesOrderID",
    "inner"
)

# 9.3 Lookup Transformation - Add product_sk from product_df based on ProductID
merged_data_with_product_sk = merged_data.join(
    product_df.select("product_id"),
    merged_data.Product_id == product_df.product_id,
    "left"
).drop(product_df.product_id)

# 9.4 DRC - Replace nulls
merged_data_validated = merged_data_with_product_sk \
    .withColumn("TaxAmt", coalesce(col("TaxAmt"), lit(0))) \
    .withColumn("Freight", coalesce(col("Freight"), lit(0))) \
    .withColumn("product_sk", coalesce(col("product_sk"), lit(-1)))  

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2715838316518993>:10
      2 merged_data = sales_order_header_sorted.join(
      3     sales_order_detail_sorted,
      4     "SalesOrderID",
      5     "inner"
      6 )
      8 # 9.3 Lookup Transformation - Add product_sk from product_df based on ProductID
      9 merged_data_with_product_sk = merged_data.join(
---> 10     product_df.select("product_id"),
     11     merged_data.Product_id == product_df.product_id,
     12     "left"
     13 ).drop(product_df.product_id)
     15 # 9.4 DRC - Replace nulls
     16 merged_data_validated = merged_data_with_product_sk \
     17     .withColumn("TaxAmt", coalesce(col("TaxAmt"), lit(0))) \
     18     .withColumn("Freight", coalesce(col("Freight"), lit(0))) \
     19     .withColumn("product_sk", coalesce(col("product_sk"), lit(-1)))

File /databricks/spark/python/pyspark/inst

In [0]:

# 9.6 Load Phase - Select columns for the fact table
fact_table = merged_data_final.select(
    col("product_sk"),
    col("SalesOrderID").alias("sales_order_id"),
    col("SalesOrderDetailID").alias("line_number"),
    col("OrderQty").alias("quantity"),
    col("UnitPrice").alias("unit_price"),
    col("standard_cost").alias("unit_cost"),
    col("TaxAmt").alias("tax_amount"),
    col("Freight").alias("freight"),
    col("extended_sales"),
    col("OrderDate").alias("created_at"),
    col("extended_cost")
)


In [0]:
product_dim.show(5)
sales_order_header.show(5)
sales_order_detail.show(5)

+----------+------------+-------------------+-------------+-------------+
|product_id|product_name|product_description|reorder_point|standard_cost|
+----------+------------+-------------------+-------------+-------------+
|         1|        null|               null|          750|          0.0|
|         2|        null|               null|          750|          0.0|
|         3|        null|               null|          600|          0.0|
|         4|        null|               null|          600|          0.0|
|       316|        null|               null|          600|          0.0|
+----------+------------+-------------------+-------------+-------------+
only showing top 5 rows

+------------+----------------+----------+----------+-----------+--------+-------+
|SalesOrderID|SalesOrderNumber| OrderDate|CustomerID|TerritoryID|  TaxAmt|Freight|
+------------+----------------+----------+----------+-----------+--------+-------+
|       43697|         SO43697|2011-05-31|     21768|       